#### Imports / setup

In [1]:
import pandas as pd
from geemap import geojson_to_ee

from whisp.src.agstack_to_gee import register_fc_and_append_to_csv, add_geo_ids_to_csv_from_lookup_csv
from whisp.src.assess_risk import whisp_stats_from_geojson_roi
from whisp.src.tidy_tables import whisp_risk
from whisp.src.utils import init_ee

from whisp.parameters.config_asr import asset_registry_base, session, token
from whisp.parameters.config_runtime import geo_id_column, out_csv, out_directory

Authentication failed. Status code: 502
Error message: <html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.14.0 (Ubuntu)</center>
</body>
</html>



In [2]:
init_ee()

#### Inputs : feature collection (without Geo IDs)
- Choose between 1) GEE asset or 2) Geojson input
- Comment out the one you dont want

In [3]:
# asset_id = "projects/ee-andyarnellgee/assets/example_asset" # asset id
# roi = ee.FeatureCollection(asset_id) # load feature collection asset

geojson_path = "../input_examples/geojson_example.geojson" # geojson file
roi = geojson_to_ee(geojson_path) # convert geojson to feature collection

In [4]:
roi # view feature collection properties

### Whisp it

In [5]:
df = whisp_stats_from_geojson_roi(geojson_path)

### Display it


In [6]:
df

,EUFO_2020,GLAD_Primary,TMF_undist,JAXA_FNF_2020,GFC_TC_2020,ESA_TC_2020,TMF_disturbed,TMF_plant,Oil_palm_Descals,Oil_palm_FDaP,...,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,RADD_before_2020,TMF_deg_after_2020,TMF_def_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index
0,0.362,0.000,0.000,1.938,0.386,1.938,0.358,0.00,0.000,0.048,...,1.552,0.00,0.000,0.000,0,0,0.000,0,0.000,0
1,16.602,14.493,15.792,16.602,16.602,16.602,0.811,0.00,0.000,0.010,...,0.000,0.00,0.000,1.111,0,0,0.000,0,0.973,1
2,0.000,0.000,0.000,1.718,0.176,29.087,0.000,31.21,30.867,9.592,...,31.020,31.21,6.401,0.000,0,0,0.000,0,0.000,2
3,5.841,0.000,6.472,20.098,11.239,20.854,9.279,0.00,0.000,0.000,...,9.043,0.00,0.000,0.000,0,0,0.000,0,0.000,3
4,0.446,0.001,0.023,7.834,5.085,6.402,0.329,0.00,0.000,0.000,...,3.194,0.00,0.000,0.000,0,0,1.602,0,0.001,4
5,1.485,0.000,0.000,3.613,1.658,3.544,0.233,0.00,0.000,0.119,...,1.955,0.00,0.000,0.000,0,0,0.000,0,0.000,5


#### Export it to CSV file
NB if exporting to Google Drive skip this and upload to your results folder 'whisp_output_table.csv'

In [7]:
#export output csv (default: '/home/sepal-user/module_results/whisp/whisp_output_table.csv')
df.to_csv(path_or_buf=out_csv,header=True,index=False)

print ("output csv: ", out_csv)

output csv:  /home/florent/module_results/src/whisp_output_table.csv


#### Add Geo ids (optional)
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- If processing doesn't complete, just rerun this cell and it should pickup from where you left.
- If registering many features you can also click dollar sign in bottom right of Sepal and under 'Sessions' heading extend the length as needed.

In [8]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="Geo_id",
    output_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)

Chosen path for temp lookup csv:/home/florent/module_results/src/temp_geo_id_lookup.csv 

Check passed: feature collection and table same size: 6 rows


KeyError: 'Geo_id'

##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [ ]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv=out_directory/"whisp_output_table.csv",
        geo_id_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        overwrite=False,
        drop_geo=False,
        debug=True)

##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [ ]:
# remove_column_from_csv(
#     csv_file="whisp_output_table_w_geo_id.csv", # this may change depending on if overwrite is on
#     column_name="system:index"
# )

##### Calculate EUDR risk category

In [9]:
# Replace with name of your CSV file 
# NB if skipped 'Add Geo ids' should set to "out_directory/whisp_output_table.csv"
csv_file_input = out_directory / "whisp_output_table.csv" 
# csv_file_input = out_directory / "whisp_output_table_w_geo_id.csv"

# import csv as a dataframe
df_w_stats = pd.read_csv(csv_file_input)

df_w_risk = whisp_risk(
    df=df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )


KeyError: 'Plot_area_ha'

##### Display table with risk columns

In [ ]:
df_w_risk

##### Export table with risk columns to csv 

In [ ]:
output_file=out_directory/'whisp_output_table_w_risk.csv' # edit as required

df_w_risk.to_csv(output_file)
